In [1]:
import pandas as pd
import psycopg2
import pandas.io.sql as psql
import geopandas as gpd
import numpy as np
import matplotlib as plt
from shapely.geometry import Point
from sklearn.neighbors import BallTree
%matplotlib inline

Conectamos con la base de datos de Twitter

In [2]:
try:
    conn = psycopg2.connect(user = "ffunes",
                            password = "location8",
                            host = "127.0.0.1",
                            port = "5432",
                            database = "elecciones_twitter")
except:
    print("No se pudo conectar a la base de datos")

In [3]:
cur = conn.cursor()

¿Cuantos Tweets tenemos?

In [173]:
pd.read_sql_query('SELECT COUNT(*) FROM tweets',con=conn)

,count
0,897531026


¿Cuantos Tweets tenemos que incluyan bounding box?

In [170]:
pd.read_sql_query('SELECT COUNT(*) FROM tweets WHERE place_bounding_box IS NOT NULL',con=conn)

,count
0,17523907


¿Cuantos Tweets tenemos que incluyan coordenadas exactas?

In [172]:
pd.read_sql_query('SELECT COUNT(*) FROM tweets WHERE coordinates IS NOT NULL',con=conn)

,count
0,1022033


¿Cuantos Tweets tenemos que incluyen un lugar de tipo ciudad?

In [4]:
pd.read_sql_query("SELECT COUNT(*) FROM tweets WHERE place_type='city'",con=conn)

,count
0,14333550


Cargamos los Tweets geolocalizados (Que incluyen un "Bounding Box") -> tweets_with_geo.
Hay que analizar el campo "coordinates" que es una coordenada precisa siempre y cuando haya tweets que contengan "coordinates" pero no "Bounding Box"

In [4]:
tweets_with_geo = gpd.GeoDataFrame.from_postgis(
    'SELECT * FROM tweets WHERE place_bounding_box IS NOT NULL LIMIT 1000000'
    , con=conn 
    , geom_col='place_bounding_box'
)

In [6]:
tweets_with_geo.head(2)

,id,user_id,date,full_text,is_reply,replied_tweet_id,is_retweet,retweeted_tweet_id,place_type,place_name,place_country,place_bounding_box,coordinates,is_quote,quoted_tweet_id
0,1184075915350794240,91810225,2019-10-15 11:57:51,Mañana de martes. en McDonald's https://t.co/r...,False,NaN,False,None,city,Merlo,Argentina,"POLYGON ((-58.85583 -34.77089, -58.64110 -34.7...",0101000020E610000025CE699A205D4DC0B0E0E1EA3E55...,None,NaN
1,1184479206362750976,335217139,2019-10-16 14:40:24,@Florenciarietto Que me vas hablar del campo c...,True,1.184179e+18,False,None,admin,Entre Ríos,Argentina,"POLYGON ((-60.75677 -34.03795, -57.79753 -34.0...",None,None,NaN


In [73]:
tweets_with_geo.place_type.value_counts()

city            810628
admin           162654
country          16761
poi               9643
neighborhood       314
Name: place_type, dtype: int64

Por ahora, solo nos importan las ciudades

In [82]:
tweets_with_geo = tweets_with_geo.loc[tweets_with_geo["place_type"] == "city", :]

In [83]:
tweets_with_geo.shape

(810628, 20)

Vemos que si hay campos que incluyen coordenada especifica que no Bounding Box, son muy pocos realmente

In [131]:
coordinates_and_not_bounding = gpd.GeoDataFrame.from_postgis(
    'SELECT * FROM tweets WHERE place_bounding_box IS NULL AND coordinates IS NOT NULL LIMIT 100000'
    , con=conn 
    , geom_col='place_bounding_box'
)

In [132]:
coordinates_and_not_bounding.shape

(2656, 15)

Pasamos a minúsculas los campos de texto que nos importan para verificar si hay correctitud entre Ciudad/Pais con las coordenadas/bounding box. Update: Intentamos sacar los acentos también

In [124]:
import unidecode

def to_lower(row, column):
    if not type(row[column] == str):
        return row[column]
        
    return unidecode.unidecode(str(row[column]).lower())

In [8]:
tweets_with_geo["place_name"] = tweets_with_geo.apply(to_lower, axis=1, column="place_name")
tweets_with_geo["place_country"] = tweets_with_geo.apply(to_lower, axis=1, column="place_country")

In [181]:
tweets_with_geo.head(2)

,id,user_id,date,full_text,is_reply,replied_tweet_id,is_retweet,retweeted_tweet_id,place_type,place_name,place_country,place_bounding_box,coordinates,is_quote,quoted_tweet_id,longitude_centroid,latitude_centroid,id_nearest,nearest,nearest_timezone
0,1184075915350794240,91810225,2019-10-15 11:57:51,Mañana de martes. en McDonald's https://t.co/r...,False,NaN,False,None,city,merlo,argentina,"POLYGON ((-58.85583 -34.77089, -58.64110 -34.7...",0101000020E610000025CE699A205D4DC0B0E0E1EA3E55...,None,NaN,-58.748464,-34.702805,1211,merlo,america/argentina/buenos_aires
4,1184422350630113281,167449597,2019-10-16 10:54:28,Quiero que llegue diciembre y poder decir: Ter...,False,NaN,False,None,city,tandil,argentina,"POLYGON ((-59.18293 -37.34801, -59.08280 -37.3...",None,None,NaN,-59.132867,-37.318615,1109,tandil,america/argentina/buenos_aires


Cargamos geonames para filtrar nombres de ciudades validos

In [9]:
usecols = [
    "geonameid",
    "name",
    "asciiname",
    "alternatenames",
    "latitude",
    "longitude",
    "population",
    "timezone"
]

dtypes = {
    "geonameid": np.int32,
    "name": str,
    "asciiname": str,
    "alternatenames": str,
    "latitude": np.float32,
    "longitude": np.float32,
    "feature class": str,
    "feature code": str,
    "country code": str,
    "cc2": str,
    "admin1 code": str,
    "admin2 code": str,
    "admin3 code": str,
    "admin4 code": str,
    "population": np.uint64,
    "elevation": np.float32,
    "dem": str,
    "timezone": str,
    "modification date": str    
}

In [10]:
geonames = pd.read_csv("../geonames/geonames.csv", dtype=dtypes, usecols=usecols)

In [11]:
geonames.head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,population,timezone
0,3038999,soldeu,soldeu,NaN,42.576881,1.66769,602,europe/andorra
1,3039154,el tarter,el tarter,"ehl tarter,эл тартер",42.579521,1.65362,1052,europe/andorra
2,3039163,sant julià de lòria,sant julia de loria,"san julia,san julià,sant julia de loria,sant j...",42.463718,1.49129,8022,europe/andorra
3,3039604,pas de la casa,pas de la casa,"pas de la kasa,пас де ла каса",42.542770,1.73361,2363,europe/andorra
4,3039678,ordino,ordino,"ordino,ao er di nuo,orudino jiao qu,ордино,オルデ...",42.556229,1.53319,3066,europe/andorra


Para cada Tweet, asignamos una ciudad en base al centroide del poligono que provee, esto lo vamos a utilizar para asignar una ciudad a aquellos Tweets que incluyen en place_name un lugar y no una ciudad, primero probamos con los datos para ver si el metodo que usamos es preciso

In [12]:
tweets_with_geo["longitude_centroid"] = tweets_with_geo.geometry.centroid.x
tweets_with_geo["latitude_centroid"] = tweets_with_geo.geometry.centroid.y

Usamos BallTree para obtener los puntos cercanos (KNN con K = 1), puede perder precisión pero es rápido para tantos datos

In [13]:
tree = BallTree(geonames[['longitude', 'latitude']].values, leaf_size=2, metric="euclidean")

In [14]:
_, tweets_with_geo['id_nearest'] = tree.query(
    tweets_with_geo[['longitude_centroid', 'latitude_centroid']].values,
    k=1,
)

In [15]:
asciiname_index = geonames.columns.get_loc('asciiname')
timezone_index = geonames.columns.get_loc('timezone')

def geonames_by_id(row):
    return geonames.iloc[row["id_nearest"], asciiname_index]

def geonames_timezone_by_id(row):
    return geonames.iloc[row["id_nearest"], timezone_index]

In [16]:
tweets_with_geo["nearest"] = tweets_with_geo.apply(geonames_by_id, axis=1)
tweets_with_geo["nearest_timezone"] = tweets_with_geo.apply(geonames_timezone_by_id, axis=1)

Separamos los distintos nombres de una ciudad y armamos un dataframe con esos nombres

In [104]:
geonames_splitted_names = (
    geonames.set_index(geonames.columns.drop('alternatenames',1).tolist())
    .alternatenames.str.split(',', expand=True)
    .stack()
    .reset_index()
    .rename(columns={0:'alternatenames'})
    .loc[:, ["alternatenames"]]
)

In [105]:
geonames_splitted_names.shape

(816602, 1)

In [106]:
geonames_splitted_names = pd.merge(
    left=geonames_splitted_names,
    right=geonames.loc[:, ["asciiname"]].rename(columns={"asciiname": "alternatenames"}),
    how='outer'
)
geonames_splitted_names = pd.merge(
    left=geonames_splitted_names,
    right=geonames.loc[:, ["name"]].rename(columns={"name": "alternatenames"}),
    how='outer'
)

In [107]:
geonames_splitted_names.shape

(6115353, 1)

In [108]:
geonames_splitted_names.drop_duplicates(inplace=True)

In [109]:
geonames_splitted_names.shape

(770143, 1)

In [111]:
geonames_splitted_names.head(10)

,alternatenames
0,ehl tarter
1,эл тартер
2,san julia
3,san julià
4,sant julia de loria
5,sant julià de lòria
6,sant-zhulija-de-lorija
7,sheng hu li ya-de luo li ya
8,сант-жулия-де-лория
9,サン・ジュリア・デ・ロリア教区


Removemos los nombres con acentos y unificamos (de la misma forma que vamos a hacer para los place_name)

In [125]:
geonames_splitted_names["alternatenames"] = geonames_splitted_names.apply(to_lower, axis=1, column="alternatenames")

In [126]:
geonames_splitted_names.drop_duplicates(inplace=True)

In [127]:
geonames_splitted_names.shape

(616740, 1)

Vemos que los Tweets incluyen lugares (ciudades) que no estan contemplados en geonames

In [128]:
tweets_with_geo.shape

(810628, 20)

In [129]:
tweets_with_geo_and_city = pd.merge(
    left=tweets_with_geo,
    right=geonames_splitted_names,
    how='inner',
    left_on=["place_name"],
    right_on=["alternatenames"],
    validate="m:1"
)

tweets_with_geo_and_city.shape

(723713, 21)

Veamos algunos ejemplos de esos lugares (Usamos el más cercano calculado, es un nombre del geonames de igual forma). Place name incluye lugares, como Abasto shopping, no son ciudades necesariamente y en algunos casos estan excluidos los nombres por un tema de acentos (Ver incluso que el Abasto está en Balvanera)

In [130]:
(tweets_with_geo
    .loc[~(tweets_with_geo.id.isin(tweets_with_geo_and_city.id)), ["place_name", "nearest", "id"]]
    .groupby(["place_name", "nearest"])
    .count().sort_values("id", ascending=False))

,,id
place_name,nearest,
villa soldati,villa lugano,10710
vicente lopez,olivos,9434
gonzalez catan,jose maria ezeiza,8612
lanus oeste,lanus,7735
almirante brown,florencio varela,6140
ciudad del libertador general san martin,general san martin,5736
"bogota, d.c.",pasca,5028
ituzaingo centro,ituzaingo,3115
embu-guacu,embu guacu,1826


Veamos con que precisión funciona BallTree para asignar ciudades a los que ya poseen un nombre de ciudad. Muy poca, veamos que pasa realmente

In [131]:
tweets_with_geo_and_city.loc[tweets_with_geo_and_city.place_name == tweets_with_geo_and_city.nearest, :].shape

(452592, 21)

Puede ser que geonames de lugares con aún mayor precisión que lo que incluye Twitter, hay que revisar, update: además hay un tema de acentos de por medio (Solucionado)

In [132]:
(tweets_with_geo_and_city
    .loc[tweets_with_geo_and_city.place_name != tweets_with_geo_and_city.nearest, ["nearest", "place_name", "id"]]
    .groupby(["nearest", "place_name"])
    .count()
    .sort_values("id", ascending=False)
    .head(10))

,,id
nearest,place_name,
colegiales,ciudad autonoma de buenos aires,126898
diadema,sao paulo,14237
nilopolis,rio de janeiro,9433
general pacheco,tigre,6385
berazategui,florencio varela,5751
belo horizonte,bello horizonte,5353
playas,guayaquil,5044
tutamandahostel,quito,4765
porto alegre,puerto alegre,3935


Revisemos si el algoritmo funciona bien para Tweets con precisión exacta

In [46]:
tweets_with_exact_geo = gpd.GeoDataFrame.from_postgis(
    'SELECT * FROM tweets WHERE coordinates IS NOT NULL LIMIT 1000000'
    , con=conn 
    , geom_col='coordinates'
)

In [47]:
tweets_with_exact_geo.shape

(1000000, 15)

In [48]:
tweets_with_exact_geo["place_name"] = tweets_with_exact_geo.apply(to_lower, axis=1, column="place_name")
tweets_with_exact_geo["place_country"] = tweets_with_exact_geo.apply(to_lower, axis=1, column="place_country")

In [49]:
tweets_with_exact_geo["longitude"] = tweets_with_exact_geo.geometry.centroid.x
tweets_with_exact_geo["latitude"] = tweets_with_exact_geo.geometry.centroid.y

In [50]:
_, tweets_with_exact_geo['id_nearest'] = tree.query(
    tweets_with_exact_geo[['longitude', 'latitude']].values,
    k=1,
)

In [51]:
tweets_with_exact_geo["nearest"] = tweets_with_exact_geo.apply(geonames_by_id, axis=1)

In [52]:
tweets_with_exact_geo_and_city = pd.merge(
    left=tweets_with_exact_geo,
    right=geonames_splitted_names,
    how='inner',
    left_on=["place_name"],
    right_on=["alternatenames"],
    validate="m:1"
)

tweets_with_exact_geo_and_city.shape

(854213, 20)

In [53]:
(tweets_with_exact_geo_and_city
    .loc[tweets_with_exact_geo_and_city.place_name == tweets_with_exact_geo_and_city.nearest, :]
    .shape)

(443179, 20)

Podemos ver que incluso con coordenadas exactas, el problema es de acentos, idioma (solucionado) y granularidad de la ubicacion provista por Twitter, a veces determina una ciudad, otras veces una provincia e incluso locales, geonames es más riguroso respecto a esto

In [54]:
(tweets_with_exact_geo_and_city
    .loc[tweets_with_exact_geo_and_city.place_name != tweets_with_exact_geo_and_city.nearest, ["nearest", "place_name", "id"]]
    .groupby(["nearest", "place_name"])
    .count()
    .sort_values("id", ascending=False)
    .head(20))

,,id
nearest,place_name,
buenos aires,ciudad autonoma de buenos aires,34441
colegiales,ciudad autonoma de buenos aires,23038
boedo,ciudad autonoma de buenos aires,21363
retiro,ciudad autonoma de buenos aires,12659
balvanera,ciudad autonoma de buenos aires,12131
belgrano,ciudad autonoma de buenos aires,6647
villa santa rita,ciudad autonoma de buenos aires,5758
general levalle,cordoba,3948
mexico city,cuauhtemoc,3591


Veamos si la ubicacion de los usuarios es un dato confiable

Tenemos en total 2 millones de usuarios activos

In [44]:
pd.read_sql_query('SELECT COUNT(*) FROM users',con=conn)

,count
0,2020784


In [29]:
users = pd.read_sql_query('SELECT * FROM users WHERE location IS NOT NULL',con=conn)

De los cuales aproximadamente 1 millón proveen su ubicacion en el perfil, veamos si es un dato confiable (Nos quedamos solo con aquellos que coincida con geonames)

In [133]:
users.shape

(1063795, 15)

In [31]:
users.head()

,id,is_private,favourites_count,followers_count,friends_count,listed_count,statuses_count,location,support,screen_name,name,loc_in_argentina,important
0,1164850459,False,1272,263,1220,1,4622,BRASIL,frentedetodos,tapa11seba,Sebastian Nardone,no_arg,False
1,565758103,False,9907,101,448,2,2242,"Belén de Escobar, Argentina",None,mforesti23,marcelo daniel,arg,True
2,111936587,False,49,273,2116,3,1011,VENEZUELA,frentedetodos,jonathanperez13,JONATHAN PEREZ,no_arg,False
3,1729550665,False,12165,1161,4810,11,19443,"Gijón, España",None,javier40434691,Javier GIJÓN,no_arg,False
4,1079263385240420352,False,1600,2,15,0,13,"Tandil, Argentina",None,FrancoL07288211,Franco López,arg,True


Unificamos la información para manejar strings

In [32]:
users["location"] = users.apply(to_lower, axis=1, column="location")

In [134]:
users_matches_geonames = pd.merge(
    left=users,
    right=geonames_splitted_names,
    how='inner',
    left_on=["location"],
    right_on=["alternatenames"],
    validate="m:1"
)

Aproximadamente 280.000 usuarios incluyen una ubicación "real", y con "real" nos referimos a que está incluida en geonames de manera exacta

In [135]:
users_matches_geonames.shape

(288136, 16)

Veamos ejemplos donde esto no ocurre

In [136]:
(users
    .loc[~(users.id.isin(users_matches_geonames.id)), ["location", "id"]]
    .groupby("location")
    .count().sort_values("id", ascending=False))

,id
location,
"buenos aires, argentina",57776
ciudad autonoma de buenos aire,22692
"cordoba, argentina",21963
"rosario, argentina",11430
"mendoza, argentina",9240
"la plata, argentina",7782
"santa fe, argentina",7345
ecuador,6597
"salta, argentina",5825


Vemos que el problema es una coma que incluye el pais, podemos excluir esto (Incluso un parseo más generico)

In [36]:
def retrieve_country_place(row):
    location = row['location'].replace("-", ",")
    location = location.replace("/", ",")
    split = location.split(',')
        
    if len(split) > 1:
        return split[0], split[-1].replace(' ', '')
        
    return None, split[-1]

In [37]:
users['place'], users['country'] = zip(*users.apply(retrieve_country_place, axis=1))

In [137]:
users_matches_geonames = pd.merge(
    left=users,
    right=geonames_splitted_names,
    how='inner',
    left_on=["place"],
    right_on=["alternatenames"],
    validate="m:1"
)

Mejoramos bastante

In [138]:
users_matches_geonames.shape

(444017, 16)

¿Que dejamos afuera ahora? Se ve que hay locations que son exclusivamente paises

In [139]:
(users
    .loc[~(users.id.isin(users_matches_geonames.id)), ["location", "id"]]
    .groupby("location")
    .count().sort_values("id", ascending=False))

,id
location,
argentina,74936
ciudad autonoma de buenos aire,22692
venezuela,22644
buenos aires,21661
ecuador,6597
mexico,5302
chile,4888
bolivia,4778
rosario,4690


Entonces, más usuarios ubicando unicamente el pais

In [140]:
pd.merge(
    left=users,
    right=geonames_splitted_names,
    how='inner',
    left_on=["country"],
    right_on=["alternatenames"],
    validate="m:1"
).shape

(702089, 16)

In [142]:
users_with_tweets = pd.merge(
    left=users_matches_geonames,
    right=tweets_with_geo,
    how='inner',
    left_on=["id"],
    right_on=["user_id"],
    validate="1:m"
)

users_with_tweets.shape

(328035, 36)

De aquellos Tweets, solo 11.000 (para los datos usados) pertenecen a usuarios unicos

Vemos que coincide más con las ubicaciones propias de los Tweets

In [163]:
locations_nearest = (users_with_tweets.groupby(["user_id", "nearest"])
    .count()
    .sort_values("user_id", ascending=False)
    .reset_index()
    .drop_duplicates("user_id", keep='first'))

users_median_location = pd.merge(
    left=users,
    right=locations_nearest.loc[:, ["user_id", "nearest"]],
    how='inner',
    left_on="id",
    right_on="user_id"
)

users_median_location.shape

(11302, 17)

In [164]:
users_median_location.loc[users_median_location["place"] == users_median_location["nearest"], :].shape

(5329, 17)

In [160]:
locations_nearest = (users_with_tweets.groupby(["user_id", "place_name"])
    .count()
    .sort_values("user_id", ascending=False)
    .reset_index()
    .drop_duplicates("user_id", keep='first'))

users_median_location = pd.merge(
    left=users,
    right=locations_nearest.loc[:, ["user_id", "place_name"]],
    how='inner',
    left_on="id",
    right_on="user_id"
)

users_median_location.shape

(11302, 17)

In [162]:
users_median_location.loc[users_median_location["place"] == users_median_location["place_name"], :].shape

(5686, 17)

Vemos otra vez que difiere, esto es porque Twitter asigna los lugares, hay garantia de precisión? Quizás la predicción de BallTree no es tan mala, incluye, en muchos casos, una ubicación más específica (Puede ser falsa también)

In [168]:
(users_with_tweets
    .drop_duplicates("id_x")
    .loc[users_with_tweets.place != users_with_tweets.nearest, ["nearest", "place", "id_x"]]
    .groupby(["nearest", "place"])
    .count()
    .sort_values("id_x", ascending=False)
    .head(20))

id_x
nearest               place                                
colegiales            buenos aires                      768
                      ciudad autonoma de buenos aires   150
jose maria ezeiza     buenos aires                      112
san miguel de tucuman tucuman                            86
general pacheco       tigre                              57
godoy cruz            mendoza                            51
villa lugano          buenos aires                       51
olivos                buenos aires                       48
general san martin    buenos aires                       46
villa nueva           mendoza                            45
berazategui           buenos aires                       45
diadema               sao paulo                          40
lanus                 buenos aires                       38
avellaneda            buenos aires                       36
moron                 buenos aires                       36
quilmes               buenos aires                       34
san isidro            buenos aires                       34
caseros               buenos aires                       33
colegiales            argentina                          33
san salvador de jujuy jujuy                              31

In [169]:
geonames.loc[geonames["asciiname"] == "tutamandahostel", :]

,geonameid,name,asciiname,alternatenames,latitude,longitude,population,timezone
49248,10277901,tutamandahostel,tutamandahostel,NaN,-0.19727,-78.497498,140000,america/guayaquil


In [134]:
cur.close()
conn.close()
print("Sesion cerrada")

Sesion cerrada
